# Programm zur Erkennung von Hunderassen

## Schritt 1: Daten laden und Aufteilen in Trainings- und Testdaten

In [1]:
# Importieren der notwendigen Bibliotheken
import os
import numpy as np
import xml.etree.ElementTree as ET
from skimage import io, color, transform
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [2]:
# Definieren der Pfade zum Datensatz und zu den Annotationen
dataset_path = 'dogs'          # Passen Sie den Pfad an
annotation_path = 'annotations'  # Passen Sie den Pfad an


In [3]:
# Funktion zum Parsen der Annotationsdateien
def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    bndboxes = []
    for obj in root.findall('object'):
        bndbox = obj.find('bndbox')
        xmin = int(float(bndbox.find('xmin').text))
        ymin = int(float(bndbox.find('ymin').text))
        xmax = int(float(bndbox.find('xmax').text))
        ymax = int(float(bndbox.find('ymax').text))
        bndboxes.append((xmin, ymin, xmax, ymax))
    return bndboxes


In [4]:
# Laden der Bilder und Extrahieren der Hundebereiche
images = []
labels = []
breeds = os.listdir(dataset_path)

for breed in breeds:
    breed_img_path = os.path.join(dataset_path, breed)
    breed_anno_path = os.path.join(annotation_path, breed)
    if os.path.isdir(breed_img_path):
        for img_name in os.listdir(breed_img_path):
            img_path = os.path.join(breed_img_path, img_name)
            annotation_name = os.path.splitext(img_name)[0]
            annotation_file = os.path.join(breed_anno_path, annotation_name)
            if not os.path.exists(annotation_file):
                continue  # Überspringe Bilder ohne Annotation
            image = io.imread(img_path)
            if image is None:
                continue
            bndboxes = parse_annotation(annotation_file)
            for bndbox in bndboxes:
                xmin, ymin, xmax, ymax = bndbox
                xmin = max(0, xmin)
                ymin = max(0, ymin)
                xmax = min(image.shape[1], xmax)
                ymax = min(image.shape[0], ymax)
                if xmin >= xmax or ymin >= ymax:
                    continue  # Ungültige Bounding Box überspringen
                cropped_image = image[ymin:ymax, xmin:xmax]
                cropped_image = transform.resize(cropped_image, (256, 256))
                images.append(cropped_image)
                labels.append(breed)


In [5]:
# Umwandeln der Labels in numerische Werte
le = LabelEncoder()
y = le.fit_transform(labels)

# Überprüfen, ob Bilder geladen wurden
if len(images) == 0:
	print("Fehler: Keine Bilder geladen. Bitte überprüfen Sie den Pfad und die Annotationsdateien.")
else:
	# Aufteilen in Trainings- und Testdaten
	X_train_img, X_test_img, y_train, y_test = train_test_split(images, y, test_size=0.25, stratify=y, random_state=42)


In [6]:
# Überprüfen der Anzahl der geladenen Bilder
print(f'Anzahl der gesamten Bilder: {len(images)}')
print(f'Anzahl der Trainingsbilder: {len(X_train_img)}')
print(f'Anzahl der Testbilder: {len(X_test_img)}')


Anzahl der gesamten Bilder: 1029
Anzahl der Trainingsbilder: 771
Anzahl der Testbilder: 258
